<h1 id="simple-chat-bot"><center>💬 Simple Chat Bot 💬</center></h1>
<i><center>Create your first Chat Bot with Python</center></i>

----

<h2 id="problem-description">📝 Problem Description</h2>

<br />

<figure>
    <img src="https://res.cloudinary.com/dte7upwcr/image/upload/blog/blog2/chatbot-o-que-e/chatbot-o-que-e-img_header.jpg" alt="Chat Bot Image" />
    <figcaption><i>Fig. 1 - Chat Bot. <sup>©</sup><a href="https://cloudinary.com" target="_blank">Cloudinary</a></i></figcaption>
</figure>

<br />

You have been hired by CUMI (*Central Unity of My Imagination*) to construct a Chat Bot model to talk with people in **English**, **Japanese** and **Portuguese**. The bot does not have a especific purpose, that is, it should just answer simple small talk questions.

You received a dataset containing more than 3k questions and answers to train the model. You can use any programming language and library, however, you have to explain each project step.

In the end, save the model in a pickle file to others be able to use it!

----

<h2 id="files-description">📁 Files Description</h2>

> **conversation.csv** - contains around 3K simple small talk questions and answers to train the model.

----

<h2 id="features">❓ Features</h2>

> **Id** - row id to identify the (*question, answer*) pair;

> **Question** - small talk question.

----

<h2 id="target-feature">🌟 Target Feature</h2>

> **Answer** - small talk answer.

----

<h2 id="metric">📏 Metric</h2>

There are inumerous metrics we can us to evaluate Chat Bot models, such as *Number of Active Users*, *Users Satisfaction Score*, *Number of Total Intections per Conversation* and *Time/Lenght of Conversation*. In this project, we will be using **Goal Completion Rate (*GCR*)**.

*GCR* measures how good the model fills its role. In our case, our model goal is to answer simple small talk questions, so, after training it, we will do some conversation tests and se how good and coherent the model answers.

----

<h2 id="limitations">🛑 Limitations</h2>

Nothing is perfect in this world and Artificial Intelligence and Data Science are not exceptions. Always there will be limitations in our projects.

The first - and most obvious - limitation we have here are the languages! Our bot will be able to understand English, Japanese and Portuguese only, so, if the model's accessibility becomes public to any person around the world, this person must now one of these three languages or use translators (like *Google Translate*) in order to interact with it.

Besides, even though our dataset contains around 3k small talk questions, it does not contain all possible questions in small talk and our model will probably not answer well some especific questions - yeah, I believe that we won't be able to ask about animes and shows spoilers. 😥

----

<h2 id="goals">🎯 Goals</h2>

> **Goal 1** - make the model answer simple  small talk questions based on our dataset;

> **Goal 2** - make the model be able to understand and answer in **English**, **Japanese** and **Portuguese**;

> **Goal 3** - save the model in pickle format.

----

<h2 id="setup">⚙️ Setup</h2>

***Tools***

> Python Version 3.9.x+;

> Jupyter Notebook;

<br />

***Libraries***

> Pandas, Numpy, Matplotlib;

> ChatterBot;

> Pickles.

----

<h2 id="acknowledgments">🎉 Acknowledgements</h2>

> Dataset by [*Kreesh Rajani*](https://www.kaggle.com/kreeshrajani) from Kaggle;

> Dataset URL: [3k Conversations Dataset for Chat Bot](https://www.kaggle.com/datasets/kreeshrajani/3k-conversations-dataset-for-chatbot).

----

In [1]:
import pandas as pd # pip install pandas
import numpy as np # pip install numpy
import matplotlib.pyplot as plt # pip install matplotlib

SEED = (4353)

np.random.seed(SEED)
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 5)

----

<h2 id="reading-the-dataset">0) Reading the Dataset</h2>

In [2]:
dataset_path = './dataset/conversation.csv'
df = pd.read_csv(dataset_path, index_col='Unnamed: 0')
df.head()

,question,answer
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


----

<h2 id="exploring-the-dataset">1) Exploring the Dataset</h2>

In [3]:
# Statistic Overview
df.describe()

,question,answer
count,3725,3725
unique,3510,3512
top,what do you mean?,what do you mean?
freq,22,22


Looking at the table above, we realize that there are duplicated questions and answers, being **what do you mean?** the most frequent one in both features. Let's take a look to all rows that has this sentence and check out whether we will drop or not them.

In [4]:
# Listing all rows containing "what do you mean?" on questions and answers
value = 'what do you mean?'
filtered_rows = df.loc[(df['question'] == value) | (df['answer'] == value)]
filtered_rows_indexes = filtered_rows.index


for (index, question, answer) in zip(
    range(len(filtered_rows))
    , filtered_rows['question']
    , filtered_rows['answer']
):
    print(f'{index}) {question} >> {answer}')

0) you're watching too much tv. >> what do you mean?
1) what do you mean? >> i mean you're wasting your life.
2) no, when i call him on his cell phone. >> what do you mean?
3) what do you mean? >> i buried him with his cell phone.
4) no. that's incomplete. >> what do you mean?
5) what do you mean? >> what's your mailing address?
6) oh, no, you don't. >> what do you mean?
7) what do you mean? >> every morning you get up late and rush off to work late.
8) which would you prefer? >> what do you mean?
9) what do you mean? >> when you die and go to heaven, they will offer you beer or cigarettes.
10) don't be ridiculous. >> what do you mean?
11) what do you mean? >> if you're going to make a wish, wish that you were really rich or famous.
12) without gravity, you would go up. >> what do you mean?
13) what do you mean? >> you would float into the sky like a balloon.
14) and lots of thieves. >> what do you mean?
15) what do you mean? >> i mean, keep your belongings close to you.
16) that's a l

Well, let's do the booring part quick. The code bellow changes the needed "what do you mean?" questions and answers to proper ones and replace them into the main dataset.

In [5]:
replacements = [
    ["you're watching too much tv." , "i am sorry, i will cut off the screen time"],
    ["what do you mean?" , "i mean you're wasting your life."],
    ["no, when i call him on his cell phone." , "why don't you try to drop to his house and talk to him"],
    ["did you see him?" , "i spot him with his cell phone."],
    ["no. that's incomplete." , "i am sorry, i am simple model and cannot give complex answers."],
    ["are you able to send emails?" , "no, i am not able"],
    ["oh, no, you don't." , "oh, yes, i do."],
    ["i always get late to job, why?" , "every morning you get up late and rush off to work late."],
    ["which would you prefer: animes or mangas?" , "mangas are better, they are not censured."],
    ["what happens when i die?" , "when you die and go to heaven, they will offer you beer or cigarettes."],
    ["don't be ridiculous." , "oh, and you cut out being cringe, okay?"],
    ["i would like..." , "if you're going to make a wish, wish that you were really rich or famous."],
    ["without gravity, you would go up." , "yes, you and i floating forever in the space."],
    ["without gravity, you would go up." , "and you would float into the sky like a balloon."],
    ["omg, there are a lots of thieves." , "hold on, i am calling the cops."],
    ["what i do? the thieves are here!" , "i mean, keep your belongings close to you."],
    ["you are funny." , "thx, you are awesome ;)"],
    ["how many times do you travel?" , "six times a year!"],
    ["where's the car?" , "in the garage"],
    ["is the car with you?" , "the car isn't here, it is probably in the garage."],
    ["we can kill two birds with one stone." , "oh yeah!"],
    ["i gotta hit the bathroom?" , "while you're using the bathroom, i'll order some food."],
    ["because i watch the other players." , "gotcha!"],
    ["what do you mean?" , "people will tell you if they have a good hand."],
    ["there's nothing in it." , "sorry, i did not see that."],
    ["what do you mean?" , "i mean, it's just coffee."],
    ["oh, no, it isn't!" , "oh, yes, it is!"],
    ["can you watch youtube?" , "i can't get a single channel that i like."],
    ["he's a dirty old man." , "do not say that, it is a bad thing."],
    ["did you have a crush?" , "i do not have a crush, but there is a woman annoying me. she's old enough to be my mother, yet she asked me out."],
    ["no, because it hurts my ears." , "sorry, i will knock it off."],
    ["what is the worst part to fly on a airplane?" , "every time we land or take off, my ears hurt so much."],
    ["have you heard something that you're not telling me?" , "nothing!"],
    ["i am worried about my job." , "are there going to be layoffs at this place?"],
    ["for me, it's the students." , "are you sure?"],
    ["what do you mean?" , "i mean i have wonderful students."],
    ["do you think the bananas fell from the sky?" , "no, absolutely not!"],
    ["so where did the bananas come from?" , "i mean, someone used their dirty hands to pick the bananas, the apples, and the oranges."],
    ["we will be house rich, but cash poor." , "do not say that, we will be house and cash rich."],
    ["we will be cash poor." , "no! our monthly payments will be too high."],
    ["that election for u.s. senator stunk." , "i cannot opine about this subject, i do not see politics."],
    ["and how do you know the elections was not fair?" , "there were more votes than voters!"],
    ["she got an 'everything' job!" , "wow, we can see that the look is on her side."],
    ["what happened to your friend that suffered a car crash?" , "a team of doctors gave her a whole new face."]
]

for (index, replacement) in zip(filtered_rows_indexes, replacements): df.iloc[index] = [replacement[0], replacement[1]]

Now, let's check out for missing values.

In [6]:
df.isnull().sum()

question    0
answer      0
dtype: int64

As far as our dataset does not contain any missing values, let's head straight to the Machine Lerning Models.

----

<h2 id="model-choosing-and-training">2) Model Choosing and Training</h2>

For this task, we will create three ChatBot models, each one for a goal language: *Portuguese*, *Japanese* and *English*, but just the last one will be trained with our dataset, the other two will be trained with a language-corpus only. 

To create these models, we will combine multiple Machine Learning Algorithms by **ChatterBot** library to generate the replies.

In [7]:
from chatterbot import ChatBot # pip install chatterbot==1.0.4
from chatterbot.trainers import ListTrainer # pip install chatterbot_corpus
from chatterbot.trainers import ChatterBotCorpusTrainer

In [8]:
# Creating the ChatterBot Model for Each Language
#
# \ name: model's name
# \ read_only: when "True", disables the ability to the model learn after being trained
# \ logic_adapters: for answers choice
#
chatterbot_portuguese_model = ChatBot(
    name='ChatterBot Portuguese Model'
    , read_only=True
    , logic_adapters=['chatterbot.logic.BestMatch']
)

chatterbot_japanese_model = ChatBot(
    name="ChatterBot Japanese Model"
    , read_only=True
    , logic_adapters=['chatterbot.logic.BestMatch']
)

chatterbot_english_model = ChatBot(
    name='ChatterBot English Model'
    , read_only=False
    , logic_adapters=['chatterbot.logic.BestMatch']
)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gabri\AppData\Roaming\nltk_data...
[nltk

In [9]:
# Training Portuguese Model with Portuguese corpus-data
portuguese_corpus_trainer = ChatterBotCorpusTrainer(chatterbot_portuguese_model)
portuguese_corpus_trainer.train('chatterbot.corpus.portuguese')

Training compliment.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training games.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training linguistic_knowledge.yml: [####################] 100%
Training money.yml: [####################] 100%
Training proverbs.yml: [####################] 100%
Training suggestions.yml: [####################] 100%
Training trivia.yml: [####################] 100%
Training unilab.yml: [####################] 100%


In [10]:
# Training Japanese Model with Japanese corpus-data
japanese_corpus_trainer = ChatterBotCorpusTrainer(chatterbot_japanese_model)
japanese_corpus_trainer.train('chatterbot.corpus.japanese')

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [11]:
# Training English Model with English corpus-data
english_corpus_trainer = ChatterBotCorpusTrainer(chatterbot_english_model)
english_corpus_trainer.train('chatterbot.corpus.english')

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [12]:
# Training English Model with Dataset
english_list_trainer = ListTrainer(chatterbot_english_model)

for row in zip(df['question'], df['answer']):
    english_list_trainer.train(row)

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

...

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [###################

List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%
List Trainer: [####################] 100%


<h2 id="testing-the-model">3) Testing the Model</h2>

Each cell is a test for a goal language. Try it, have fun and consider that the answers will not be accurate like *ChatGPT* since our dataset is too tiny!!

In [13]:
# Generic function to test the models
def test_model(model):
    """
    Receives a ChatterBot model as a parameter and request the user input.
For each input, the bot answer is printed to the screen 'till the user tipe 'c'
without quotes.
    """
    question = input('Your question:')
    
    while question != 'c':
        print(model.get_response(question))
        print('----')
        question = input()

In [14]:
# Portuguese Model Test
test_model(chatterbot_portuguese_model)

Your question:Oi
Como vai você?
----
tudo bem e você?
também
----
c


In [16]:
# Japanese Model Test
test_model(chatterbot_japanese_model)

Your question:こんにちは
こんにちは
----
お元気ですか
元気です。
----
c


In [17]:
# English Model Test
test_model(chatterbot_english_model)

Your question:Hello
Hi
----
what's up?
nothing, how about you?
----
c


You can wonder with this tiny tests: "Woow, it really worked!!" but I don't say so...

Remember, since our dataset and the library language-corpus are so tiny, our models won't give accurate responses 100% of the time, in reality, it'll give us more innacurate responses than accurate ones and, sometime, can even change languages to reply you.

However, since this notebook goal is to give you knowledge to learn how to create chat bots, let's hop into saving the models in pickle file format.

<h2 id="saving-the-models">4) Saving the Models</h2>

In [18]:
import pickle

In [19]:
# Saving the models
pickle.dump(chatterbot_portuguese_model, open('portuguese_model.pkl', 'wb'))
pickle.dump(chatterbot_japanese_model, open('japanese_model.pkl', 'wb'))
pickle.dump(chatterbot_english_model, open('english_model.pkl', 'wb'))

TypeError: cannot pickle '_thread._local' object

----

See ya in the next notebook!! 👋

<h2 id="reach-me">📫 Reach Me</h2>

> Email: csfelix08@gmail.com

> Linkedin: [linkedin.com/in/csfelix/](https://linkedin.com/in/csfelix/)

> Portfolio: [CSFelix.io](https://csfelix.github.io)

> Kaggle: [DSFelix](https://www.kaggle.com/dsfelix)